# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Feature Pipeline</span>

## 🗒️ This notebook is divided into the following sections:

1. Fetch Feature Groups. 
2. Parse Data.
3. Feature Group Insertion.

### <span style='color:#ff5f27'> 📝 Imports

In [1]:
import datetime
import time
import pandas as pd
import json

from features import air_quality
from functions.parse_air_quality import get_aqi_data_from_open_meteo
from functions.parse_weather import get_weather_data_from_open_meteo
from functions.common_functions import *


import warnings
warnings.filterwarnings("ignore")

In [2]:
# Opening the 'target_cities.json' file in read mode using the 'with' statement
with open('target_cities.json') as json_file:
    # Loading the JSON data from the file and storing it in the 'target_cities' variable
    target_cities = json.load(json_file)

In [3]:
# Getting the current date
today = datetime.date.today()

# Displaying the current date and its string representation
today, str(today)

(datetime.date(2024, 3, 21), '2024-03-21')

### <span style="color:#ff5f27;"> 🔮 Connecting to Hopsworks Feature Store </span>

In [4]:
import hopsworks

project = hopsworks.login()
fs = project.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.

Multiple projects found. 

	 (1) Iriss_solab
	 (2) soll



Enter project to access:  2



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/22689
Connected. Call `.close()` to terminate connection gracefully.


In [5]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

## <span style='color:#ff5f27'> 🌫 Filling gaps in Air Quality data (PM2.5)</span>

In [6]:
# Read data from feature groups
df_air_quality = air_quality_fg.read()
df_weather = weather_fg.read()

Finished: Reading data from Hopsworks, using Hive (297.64s) 


Finished: Reading data from Hopsworks, using Hive (56.75s) 


In [25]:
# Extracting the "date" and "city_name" columns from the 'df_air_quality' DataFrame
# Grouping the data by "city_name" and finding the maximum date for each city
last_dates_aq = df_air_quality[["date", "city_name"]].groupby("city_name").max()

# Converting the date values to string format for consistency
last_dates_aq.date = last_dates_aq.date.astype(str)

# Creating a dictionary with city names as keys and their corresponding last updated date as values
last_dates_aq = last_dates_aq.to_dict()["date"]

In [28]:
# Accessing the last updated date for the city of Paris
paris_last_date = last_dates_aq.get("Paris", "Not available")

# Accessing the last updated date for the city of Columbus
columbus_last_date = last_dates_aq.get("Columbus", "Not available")

# Printing the results
print("⛳️ Last update for Paris:", paris_last_date)
print("⛳️ Last update for Columbus:", columbus_last_date)

⛳️ Last update for Paris: 2023-04-11
⛳️ Last update for Columbus: 2022-12-31


In [30]:
last_dates_aq

{'Albuquerque': '2022-12-31',
 'Amsterdam': '2023-04-11',
 'Athina': '2023-04-11',
 'Atlanta': '2022-12-31',
 'Bellevue-SE 12th St': '2023-04-03',
 'Berlin': '2023-04-11',
 'Chicago': '2022-12-31',
 'Columbus': '2022-12-31',
 'DARRINGTON - FIR ST (Darrington High School)': '2023-04-03',
 'Dallas': '2022-12-31',
 'Denver': '2022-12-31',
 'Gdansk': '2023-04-11',
 'Houston': '2022-12-31',
 'KENT - JAMES & CENTRAL': '2023-04-03',
 'Kraków': '2023-04-11',
 'LAKE FOREST PARK TOWNE CENTER': '2023-04-03',
 'London': '2023-04-11',
 'Los Angeles': '2022-12-31',
 'MARYSVILLE - 7TH AVE (Marysville Junior High)': '2023-04-03',
 'Madrid': '2023-04-11',
 'Marseille': '2023-04-11',
 'Milano': '2023-04-11',
 'München': '2023-04-11',
 'NORTH BEND - NORTH BEND WAY': '2023-04-03',
 'Napoli': '2023-04-11',
 'New York': '2022-12-31',
 'Paris': '2023-04-11',
 'Phoenix-Mesa': '2022-12-31',
 'SEATTLE - BEACON HILL': '2023-04-03',
 'SEATTLE - DUWAMISH': '2023-04-03',
 'SEATTLE - SOUTH PARK #2': '2023-04-03',
 '

In [34]:
today, datetime.timedelta(days=28)

(datetime.date(2024, 3, 21), datetime.timedelta(days=28))

In [32]:
datetime.datetime.strptime('2022-08-02', '%Y-%m-%d').date()

datetime.date(2022, 8, 2)

In [35]:
for city, date in last_dates_aq.items():
    city_last_date = datetime.datetime.strptime(date, "%Y-%m-%d").date()
    if (today - city_last_date) <= datetime.timedelta(days=28):
        last_dates_aq[city] = (city_last_date - datetime.timedelta(days=28)).strftime("%Y-%m-%d")

### <span style='color:#ff5f27'>  🧙🏼‍♂️ Parsing PM2.5 data

In [37]:
# Storing the current time as the start time of the cell execution
start_of_cell = time.time()

# Creating an empty DataFrame to store raw air quality data
df_aq_raw = pd.DataFrame()

# Iterating through continents and cities in the 'target_cities' dictionary
for continent in target_cities:
    for city_name, coords in target_cities[continent].items():
        # Retrieving air quality data using the 'get_aqi_data_from_open_meteo' function
        # with specified parameters such as city name, coordinates, start date, and end date
        df_ = get_aqi_data_from_open_meteo(
            city_name=city_name,
            coordinates=coords,
            start_date=last_dates_aq[city_name],
            end_date=str(today)
        )
        
        # Concatenating the retrieved data with the existing 'df_aq_raw' DataFrame
        # and resetting the index to ensure proper alignment
        df_aq_raw = pd.concat([df_aq_raw, df_]).reset_index(drop=True)

# Storing the current time as the end time of the cell execution
end_of_cell = time.time()

# Printing information about the execution, including the time taken
print("-" * 64)
print(f"Parsed new PM2.5 data for ALL locations up to {str(today)}.")
print(f"Took {round(end_of_cell - start_of_cell, 2)} sec.\n")

Processed PM2_5 for Amsterdam since 2023-04-11 till 2024-03-21.
Took 4.23 sec.

Processed PM2_5 for Athina since 2023-04-11 till 2024-03-21.
Took 3.26 sec.

Processed PM2_5 for Berlin since 2023-04-11 till 2024-03-21.
Took 2.95 sec.

Processed PM2_5 for Gdansk since 2023-04-11 till 2024-03-21.
Took 3.46 sec.

Processed PM2_5 for Kraków since 2023-04-11 till 2024-03-21.
Took 6.6 sec.

Processed PM2_5 for London since 2023-04-11 till 2024-03-21.
Took 4.17 sec.

Processed PM2_5 for Madrid since 2023-04-11 till 2024-03-21.
Took 2.02 sec.

Processed PM2_5 for Marseille since 2023-04-11 till 2024-03-21.
Took 3.0 sec.

Processed PM2_5 for Milano since 2023-04-11 till 2024-03-21.
Took 1.84 sec.

Processed PM2_5 for München since 2023-04-11 till 2024-03-21.
Took 3.57 sec.

Processed PM2_5 for Napoli since 2023-04-11 till 2024-03-21.
Took 5.49 sec.

Processed PM2_5 for Paris since 2023-04-11 till 2024-03-21.
Took 1.09 sec.

Processed PM2_5 for Sevilla since 2023-04-11 till 2024-03-21.
Took 3.47 

In [38]:
df_aq_raw.tail(3)

,city_name,date,pm2_5
17253,Tulalip-Totem Beach Rd,2024-03-19,7.8
17254,Tulalip-Totem Beach Rd,2024-03-20,9.6
17255,Tulalip-Totem Beach Rd,2024-03-21,8.8


### <span style="color:#ff5f27;">🛠 Feature Engineering PM2.5</span>

In [39]:
# Converting the 'date' column in the 'df_aq_update' DataFrame to datetime format
df_aq_raw['date'] = pd.to_datetime(df_aq_raw['date'])

In [40]:
# Applying a feature engineering function 'feature_engineer_aq' to the 'df_aq_update' DataFrame
df_aq_update = air_quality.feature_engineer_aq(df_aq_raw)

# Dropping rows with missing values in the 'df_aq_update' DataFrame
df_aq_update = df_aq_update.dropna()

df_aq_update.tail(3)

,city_name,date,pm2_5,pm_2_5_previous_1_day,pm_2_5_previous_2_day,pm_2_5_previous_3_day,pm_2_5_previous_4_day,pm_2_5_previous_5_day,pm_2_5_previous_6_day,pm_2_5_previous_7_day,...,exp_std_28_days,year,day_of_month,month,day_of_week,is_weekend,sin_day_of_year,cos_day_of_year,sin_day_of_week,cos_day_of_week
17253,Los Angeles,2024-03-21,33.5,34.0,25.5,20.8,28.0,21.5,15.1,12.7,...,8.099297,2024,21,3,3,0,0.984474,0.175531,0.433884,-0.900969
17254,Milano,2024-03-21,45.3,38.5,46.4,27.7,43.4,23.6,16.8,46.6,...,21.733977,2024,21,3,3,0,0.984474,0.175531,0.433884,-0.900969
17255,Kraków,2024-03-21,75.3,59.8,34.4,17.0,13.1,22.6,56.8,57.3,...,16.678522,2024,21,3,3,0,0.984474,0.175531,0.433884,-0.900969


In [41]:
# Checking the total number of missing values in the 'df_aq_update' DataFrame
df_aq_update.isna().sum().sum()

0

In [42]:
# Retrieving the dimensions (number of rows and columns) of the 'df_aq_update' DataFrame
df_aq_update.shape

(15994, 31)

## <span style='color:#ff5f27'> 🌦 Filling gaps in Weather data</span>

In [43]:
# Extracting the "date" and "city_name" columns from the 'df_weather' DataFrame
# Grouping the data by "city_name" and finding the maximum date for each city
last_dates_weather = df_weather[["date", "city_name"]].groupby("city_name").max()

# Converting the date values to string format for consistency
last_dates_weather.date = last_dates_weather.date.astype(str)

# Creating a dictionary with city names as keys and their corresponding last updated date as values
last_dates_weather = last_dates_weather.to_dict()["date"]

### <span style='color:#ff5f27'>  🧙🏼‍♂️ Parsing Weather data

In [44]:
# Storing the current time as the start time of the cell execution
start_of_cell = time.time()

# Creating an empty DataFrame to store raw weather data
df_weather_update = pd.DataFrame()

# Iterating through continents and cities in the 'target_cities' dictionary
for continent in target_cities:
    for city_name, coords in target_cities[continent].items():
        # Retrieving weather data using the 'get_weather_data_from_open_meteo' function
        # with specified parameters such as city name, coordinates, start date, end date, and forecast flag
        df_ = get_weather_data_from_open_meteo(
            city_name=city_name,
            coordinates=coords,
            start_date=last_dates_weather[city_name],
            end_date=str(today),
            forecast=True,
        )
        
        # Concatenating the retrieved data with the existing 'df_weather_update' DataFrame
        # and resetting the index to ensure proper alignment
        df_weather_update = pd.concat([df_weather_update, df_]).reset_index(drop=True)

# Dropping rows with missing values in the 'df_weather_update' DataFrame
df_weather_update.dropna(inplace=True)

# Storing the current time as the end time of the cell execution
end_of_cell = time.time()

# Printing information about the execution, including the time taken
print("-" * 64)
print(f"Parsed new weather data for ALL cities up to {str(today)}.")
print(f"Took {round(end_of_cell - start_of_cell, 2)} sec.\n")

Parsed weather for Amsterdam since 2023-04-13 till 2024-03-21.
Took 3.76 sec.

Parsed weather for Athina since 2023-04-13 till 2024-03-21.
Took 5.54 sec.

Parsed weather for Berlin since 2023-04-13 till 2024-03-21.
Took 3.39 sec.

Parsed weather for Gdansk since 2023-04-13 till 2024-03-21.
Took 3.31 sec.

Parsed weather for Kraków since 2023-04-13 till 2024-03-21.
Took 2.83 sec.

Parsed weather for London since 2023-04-13 till 2024-03-21.
Took 2.99 sec.

Parsed weather for Madrid since 2023-04-13 till 2024-03-21.
Took 3.65 sec.

Parsed weather for Marseille since 2023-04-13 till 2024-03-21.
Took 3.13 sec.

Parsed weather for Milano since 2023-04-13 till 2024-03-21.
Took 3.02 sec.

Parsed weather for München since 2023-04-13 till 2024-03-21.
Took 3.06 sec.

Parsed weather for Napoli since 2023-04-13 till 2024-03-21.
Took 3.01 sec.

Parsed weather for Paris since 2023-04-13 till 2024-03-21.
Took 2.86 sec.

Parsed weather for Sevilla since 2023-04-13 till 2024-03-21.
Took 2.82 sec.

Parse

In [45]:
# Converting the 'date' column in the 'df_aq_update' DataFrame to datetime format
df_aq_update.date = pd.to_datetime(df_aq_update.date)

# Converting the 'date' column in the 'df_weather_update' DataFrame to datetime format
df_weather_update.date = pd.to_datetime(df_weather_update.date)

# Creating a new column 'unix_time' in 'df_aq_update' by applying the 'convert_date_to_unix' function
df_aq_update["unix_time"] = df_aq_update["date"].apply(convert_date_to_unix)

# Creating a new column 'unix_time' in 'df_weather_update' by applying the 'convert_date_to_unix' function
df_weather_update["unix_time"] = df_weather_update["date"].apply(convert_date_to_unix)

In [46]:
# Converting the 'date' column in the 'df_aq_update' DataFrame to string format
df_aq_update.date = df_aq_update.date.astype(str)

# Converting the 'wind_direction_dominant' column in the 'df_weather_update' DataFrame to integer format
df_weather_update.wind_direction_dominant = df_weather_update.wind_direction_dominant.astype('int')

# Converting the 'date' column in the 'df_weather_update' DataFrame to string format
df_weather_update.date = df_weather_update.date.astype(str)
df_weather_update.tail(3)

,city_name,date,temperature_max,temperature_min,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,wind_speed_max,wind_gusts_max,wind_direction_dominant,unix_time
15477,Tulalip-Totem Beach Rd,2024-03-19,17.7,5.1,0.0,0.0,0.0,0.0,8.8,18.4,345,1710795600000
15478,Tulalip-Totem Beach Rd,2024-03-20,17.1,7.2,0.0,0.0,0.0,0.0,12.6,28.4,190,1710882000000
15479,Tulalip-Totem Beach Rd,2024-03-21,13.7,6.8,0.4,0.6,0.0,1.0,12.1,22.7,159,1710968400000


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [47]:
# Insert new data
air_quality_fg.insert(df_aq_update)

Uploading Dataframe: 0.00% |          | Rows 0/15994 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/22689/jobs/named/air_quality_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x15df1f400>, None)

In [48]:
# Insert new data
weather_fg.insert(df_weather_update)

Uploading Dataframe: 0.00% |          | Rows 0/7335 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/22689/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x15df40550>, None)

---
## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will create a feature view, create a training dataset, train a model and save it in the Hopsworks Model Registry.